In [ ]:
pip install requests
pip install lxml
pip install pandas
pip install openpyxl
pip install cobra
pip install cobrakbase
# for windows only, download the file from https://www.lfd.uci.edu/~gohlke/pythonlibs/#pyeda firstly
# pip install pyeda-0.28.0-cp37-cp37m-win_amd64.whl  
pip install modelseedpy


In [1]:
org_name = 'mko'
org_full_name = 'Methylomonas_koyamae'

In [1]:
org_name = 'mbry'
org_full_name = 'Methylocystis_bryophila'

# build model from modelseed

## Initialize new model

In [2]:
import cobra

new_model = cobra.Model(org_full_name)
new_model

Name,Methylomonas_koyamae
Memory address,275915d1f40
Number of metabolites,0
Number of reactions,0
Number of genes,0
Number of groups,0
Objective expression,0
Compartments,


## KEGG ID mapping to modelseed

In [3]:
import pandas as pd

#org_name = 'mbry'
df_reaction = pd.read_excel(org_name+'_model\\'+org_name+'_clean_kegg_reaction.xlsx', index_col=1)  
titlename= ['entry_name','pathway_title','reaction_type','reaction_substrates','reaction_products']
df_keggID_modelseedID = pd.read_csv('reaction_keggID_modelseedID.txt', sep='\t')
mapping_reaction = pd.merge(df_reaction,df_keggID_modelseedID,how='left',on='Kegg_ID').drop_duplicates(subset=['Kegg_ID'], keep='first')[['ModelSeed_ID','Kegg_ID']+titlename]
mapping_reaction.to_excel(org_name+'_model\\'+org_name+'_mapping_reaction.xlsx', sheet_name='Sheet1', header=True)

print('done!')

done!


## add c0 reactions from ModelSeed Database

In [3]:
def add_ms_reaction(
    model,
    rxn_id,
    modelseed,
    compartment = 'c0',
    direction="forward",
):  # Xinli modified from modelseedpy.core.mseditorapi

    modelseed_reaction = modelseed.get_seed_reaction(rxn_id)
    reaction_stoich = modelseed_reaction.cstoichiometry
    cobra_reaction = cobra.Reaction(rxn_id+'_'+ compartment)
    cobra_reaction.name = str(modelseed_reaction.data["name"])+'_'+compartment

    metabolites_to_add = {}
    for metabolite, stoich in reaction_stoich.items():
        id = metabolite[0]
        compound = modelseed.get_seed_compound(id).data
        if int(metabolite[1]) == 0:
            compartment_string = 'c0'
        elif int(metabolite[1]) == 1:
            compartment_string = 'e0'
        else:
            compartment_string = ''
            print(str(id)+' compartment wrong')

        metabolites_to_add[
            cobra.Metabolite(
                id+'_'+compartment_string, name=compound["name"]+'_'+compartment_string, compartment=compartment_string, 
                # formula = compound['formula']
            )
        ] = stoich
    cobra_reaction.add_metabolites(metabolites_to_add)
    cobra_reaction.reaction
    if direction == "reversible":
        cobra_reaction.lower_bound = -1000
    elif direction == "backward":
        cobra_reaction.lower_bound = -1000
        cobra_reaction.upper_bound = 0

    model.add_reactions([cobra_reaction])

In [4]:
import cobra
import pandas as pd

mapping_reaction = pd.read_excel(org_name+'_model\\'+org_name+'_mapping_reaction.xlsx', index_col=0)  
modelseed_id_list = list(mapping_reaction['ModelSeed_ID'].dropna(axis = 0, how = 'any'))  # reaction ID that need to be added


import modelseedpy
modelseed_path = 'C:\\Users\\vickenlee\\ModelSEEDDatabase'
modelseed = modelseedpy.biochem.from_local(modelseed_path)

print('======================== \n %d reactions begin to added \n'%len(modelseed_id_list))

i,j = 0,0
for rxn_id in modelseed_id_list:
    reaction_type = mapping_reaction.query('ModelSeed_ID == "'+ rxn_id +'"')['reaction_type'].values[0]
    if reaction_type == 'reversible':
        direction = 'reversible'
    elif reaction_type == 'irreversible':
        direction = 'forward'
    try:
        add_ms_reaction(new_model, rxn_id = rxn_id, modelseed = modelseed, compartment = 'c0', direction = direction)
        print(rxn_id+' added')
        i+=1
    except:
        print(rxn_id+' added wrong !!!')
        j+=1

print('\n %d / %d reactions added successfully \n ====================='%(i,i+j))

for metabolite in new_model.metabolites:
    metaid = [meta for meta in metabolite.id.split('_') if meta.startswith('cpd')]
    if metaid:
        ms_metabolite = modelseed.get_seed_compound(metaid[0])
        if  ms_metabolite.formula == ms_metabolite.formula:
            metabolite.formula = ms_metabolite.formula
        metabolite.charge = ms_metabolite.data['charge']

# cobra.io.write_sbml_model(new_model,filename=org_name+'_model\\'+org_name+'_add_c0_from_modelseed.xml')

 804 reactions begin to added 

rxn15116 added
rxn00536 added
rxn00543 added
rxn00011 added
rxn02342 added
rxn01871 added
rxn00148 added
rxn00459 added
rxn00781 added
rxn00747 added
rxn15494 added
rxn15364 added
rxn00704 added
rxn02380 added
rxn01169 added
rxn01977 added
rxn15249 added
rxn15694 added
rxn01100 added
rxn15314 added
rxn06120 added
rxn15989 added
rxn13974 added
rxn00175 added
rxn01106 added
rxn15298 added
rxn00151 added
rxn00147 added
rxn00441 added
rxn02376 added
rxn01872 added
rxn00285 added
rxn00199 added
rxn00505 added
rxn01387 added
rxn00799 added
rxn01388 added
rxn00974 added
rxn00256 added
rxn00248 added
rxn00250 added
rxn06109 added
rxn03884 added
rxn00770 added
rxn01200 added
rxn00777 added
rxn01116 added
rxn15271 added
rxn01115 added
rxn01476 added
rxn01975 added
rxn01477 added
rxn01107 added
rxn29919 added
rxn00778 added
rxn08647 added
rxn01187 added
rxn03643 added
rxn16622 added
rxn11040 added
rxn00213 added
rxn00211 added
rxn01042 added
rxn15081 added
rxn15270

#### check balance

In [5]:
def check_balance(reaction,H_metabolite=None):
    try:
        feedback = reaction.check_mass_balance()
        if feedback :
            print(reaction.id + ': ' + str(feedback))
            print('      '+ reaction.reaction)
            if H_metabolite:
                if 'H' in feedback and feedback.get('H') == feedback.get('charge'):
                    H_to_add={H_metabolite:-1*feedback.get('H')}
                    reaction.add_metabolites(H_to_add)
                    print('      H is added')
                    return(check_balance(reaction,H_metabolite))
            return int(0)
        else:
            #print(reaction.id + ': banlance')
            #print('      '+ reaction.reaction)
            return int(1)
    except:
        print(reaction.id + ': error')
        print('      '+ reaction.reaction)
        return int(0)



import cobra
import pandas as pd

H_metabolite = new_model.metabolites.get_by_id('cpd00067_c0')

i=0
for reaction in new_model.reactions:
    i=i+check_balance(reaction,H_metabolite)
print('After add H+, %d / %d reactions are balanced'%(i,len(new_model.reactions)))

cobra.io.write_sbml_model(new_model,filename=org_name+'_model\\'+org_name+'_add_c0_from_modelseed.xml')

rxn15314_c0: {'charge': -2.0, 'H': -1.0, 'O': 3.0, 'P': 1.0}
      cpd19001_c0 --> cpd19006_c0
rxn06120_c0: {'charge': -2.0, 'H': -1.0, 'O': 3.0, 'P': 1.0}
      cpd00190_c0 --> cpd00863_c0
rxn15989_c0: {'H': -2.0}
      cpd00363_c0 + 2.0 cpd19499_c0 <=> cpd00071_c0 + 2.0 cpd19500_c0
rxn13974_c0: {'charge': 2.0}
      cpd00011_c0 + cpd00022_c0 + cpd00067_c0 + 2.0 cpd11620_c0 <=> cpd00010_c0 + cpd00020_c0 + 2.0 cpd11621_c0
rxn07578_c0: {'charge': -2.0, 'C': 11.0, 'H': 21.0, 'O': 7.0, 'N': 2.0, 'P': 1.0}
      cpd14939_c0 <=> cpd00001_c0 + cpd14940_c0
rxn07576_c0: {'charge': -1.0}
      cpd00067_c0 + cpd11476_c0 + cpd11492_c0 --> cpd00011_c0 + cpd11493_c0 + cpd14938_c0
rxn07577_c0: {'charge': 2.0, 'C': -11.0, 'H': -21.0, 'N': -2.0, 'O': -7.0, 'P': -1.0}
      cpd00005_c0 + cpd00067_c0 + cpd14938_c0 <=> cpd00006_c0 + cpd14939_c0
rxn05350_c0: {'charge': -1.0}
      cpd00067_c0 + cpd11472_c0 + cpd11492_c0 --> cpd00011_c0 + cpd11490_c0 + cpd11493_c0
rxn05336_c0: {'charge': -3.0}
      cpd000

#### print unbalanced

In [6]:
for reaction in new_model.reactions:
    if check_balance(reaction)==0:
        print('           https://modelseed.org/solr/reactions/select?wt=json&q=id:'+reaction.id.replace('_c0',''))
        for meta, sto in reaction.metabolites.items():
            print('           '+meta.id+'    '+ str(meta.formula)+'    '+str(meta.charge))

        #new_model.metabolites.get_by_id('C00042_c0').charge


rxn15314_c0: {'charge': -2.0, 'H': -1.0, 'O': 3.0, 'P': 1.0}
      cpd19001_c0 --> cpd19006_c0
           https://modelseed.org/solr/reactions/select?wt=json&q=id:rxn15314
           cpd19001_c0    C6H12O6    0
           cpd19006_c0    C6H11O9P    -2
rxn06120_c0: {'charge': -2.0, 'H': -1.0, 'O': 3.0, 'P': 1.0}
      cpd00190_c0 --> cpd00863_c0
           https://modelseed.org/solr/reactions/select?wt=json&q=id:rxn06120
           cpd00190_c0    C6H12O6    0
           cpd00863_c0    C6H11O9P    -2
rxn15989_c0: {'H': -2.0}
      cpd00363_c0 + 2.0 cpd19499_c0 <=> cpd00071_c0 + 2.0 cpd19500_c0
           https://modelseed.org/solr/reactions/select?wt=json&q=id:rxn15989
           cpd19499_c0    None    0
           cpd19500_c0    None    0
           cpd00363_c0    C2H6O    0
           cpd00071_c0    C2H4O    0
rxn13974_c0: {'charge': 2.0}
      cpd00011_c0 + cpd00022_c0 + cpd00067_c0 + 2.0 cpd11620_c0 <=> cpd00010_c0 + cpd00020_c0 + 2.0 cpd11621_c0
           https://modelseed.org/solr

In [7]:
new_model

Name,Methylomonas_koyamae
Memory address,275915d1f40
Number of metabolites,913
Number of reactions,804
Number of genes,0
Number of groups,0
Objective expression,0
Compartments,c0


## add bio reaction from Drift model

In [2]:
import cobra

#drift_model_path = org_name+'_model\\ModelSeed_model_DSMZ_21852.xml'
drift_model_path = org_name+'_model\\ModelSeed_model_'+org_full_name+'.xml'
drift_model = cobra.io.read_sbml_model(drift_model_path)
drift_model


Name,Metabolic_model_Methylocystis_bryophila
Memory address,1fb9e362340
Number of metabolites,1160
Number of reactions,1205
Number of genes,837
Number of groups,0
Objective expression,1.0*bio1_biomass - 1.0*bio1_biomass_reverse_6e711
Compartments,"c0, e0"


In [48]:
new_model = cobra.io.read_sbml_model(org_name+'_model\\'+org_name+'_add_c0_from_modelseed.xml')

drift_bioreaction = drift_model.reactions.get_by_id('bio1_biomass')
cobra_bioreaction = cobra.Reaction('rxnbiomass')
cobra_bioreaction.name = 'GramNegativeBiomass'

metabolites_to_add = {}
for metabolite, stoich in drift_bioreaction.metabolites.items():
    id = metabolite.id.replace('_c0','')
    ms_compound = modelseed.get_seed_compound(id)
    charge = ms_compound.data['charge']
    
    if  ms_compound.formula == ms_compound.formula:
        metabolites_to_add[
            cobra.Metabolite(id+'_c0', name=ms_compound.name+'_c0', compartment='c0',formula = ms_compound.formula ,charge=charge)
        ] = stoich
    else:
        metabolites_to_add[
            cobra.Metabolite(id+'_c0', name=ms_compound.name+'_c0', compartment='c0',charge=charge)
        ] = stoich

    
cobra_bioreaction.add_metabolites(metabolites_to_add)

new_model.add_reactions([cobra_bioreaction])
new_model.objective = new_model.problem.Objective(cobra_bioreaction.flux_expression, direction='max')

cobra.io.write_sbml_model(new_model,filename=org_name+'_model\\'+org_name+'_add_bio_from_modelseed.xml')

No objective coefficients in model. Unclear what should be optimized


## export to excel

In [1]:
# conda install <libsbml>
# pip install python-libsbml
import libsbml

r = libsbml.SBMLReader()
'''
doc = r.readSBML(
    ".\\5GB1_base_fermentation.xml"
)
'''
doc = r.readSBML(
    ".\\mbry_model\\mbry_add_bio_from_modelseed.xml"
)

m = doc.getModel()

for n in range(0, m.getNumReactions()):
    r = m.getReaction(n)
    # print reaction id
    print(
        #"Reaction ",
        r.getId(),
        ": ",
        end=""
    )

    # look at reactants
    numReactants = r.getNumReactants()
    if (numReactants > 1):
        s = m.getSpecies(r.getReactant(0).getSpecies())
        print(
            r.getReactant(0).getStoichiometry(),
            " ",
            s.getName(),
            " ",
            end=""
        )
        for k in range(1, numReactants):
            # get species referred to by the reaction
            s = m.getSpecies(r.getReactant(k).getSpecies())
            print(
                "+ ",
                r.getReactant(k).getStoichiometry(),
                " ",
                s.getName(),
                " ",
                end=""
            )
    elif (numReactants == 1):
        # get species referred to by the reaction
        s = m.getSpecies(r.getReactant(0).getSpecies())
        print(
            r.getReactant(0).getStoichiometry(),
            " ",
            s.getName(),
            " ",
            end=""
        )

    if (r.getReversible() == True):
        print("<=> ", end="")
    else:
        print("=> ", end="")

# look at products
    numProducts = r.getNumProducts()
    if (numProducts > 1):
        s = m.getSpecies(r.getProduct(0).getSpecies())
        print(
            r.getProduct(0).getStoichiometry(),
            " ",
            s.getName(),
            " ",
            end=""
        )
        for k in range(1, numProducts):
            # get species referred to by the reaction
            s = m.getSpecies(r.getProduct(k).getSpecies())
            print(
                "+ ",
                r.getProduct(k).getStoichiometry(),
                " ",
                s.getName(),
                " ",
                end=""
            )
    elif (numProducts == 1):
        # get species referred to by the reaction
        s = m.getSpecies(r.getProduct(0).getSpecies())
        print(
            r.getProduct(0).getStoichiometry(),
            " ",
            s.getName(),
            " ",
            end=""
        )
    print("\n")


R_rxn15116_c0 : 1.0   beta-D-Fructose 1,6-bisphosphate_c0  <=> 1.0   Glycerone-phosphate_c0  +  1.0   Glyceraldehyde3-phosphate_c0  

R_rxn00506_c0 : 1.0   H2O_c0  +  1.0   NAD_c0  +  1.0   Acetaldehyde_c0  <=> 1.0   NADH_c0  +  1.0   Acetate_c0  +  2.0   H+_c0  

R_rxn00536_c0 : 1.0   NADP_c0  +  1.0   Ethanol_c0  <=> 1.0   NADPH_c0  +  1.0   H+_c0  +  1.0   Acetaldehyde_c0  

R_rxn00543_c0 : 1.0   NAD_c0  +  1.0   Ethanol_c0  <=> 1.0   NADH_c0  +  1.0   H+_c0  +  1.0   Acetaldehyde_c0  

R_rxn00011_c0 : 1.0   CO2_c0  +  1.0   2-Hydroxyethyl-ThPP_c0  => 1.0   Pyruvate_c0  +  1.0   TPP_c0  +  1.0   H+_c0  

R_rxn02342_c0 : 1.0   Lipoamide_c0  +  1.0   2-Hydroxyethyl-ThPP_c0  => 1.0   S-Acetyldihydrolipoamide_c0  +  1.0   TPP_c0  

R_rxn01871_c0 : 1.0   Acetyl-CoA_c0  +  1.0   Dihydrolipoamide_c0  <=> 1.0   CoA_c0  +  1.0   S-Acetyldihydrolipoamide_c0  

R_rxn00148_c0 : 1.0   ATP_c0  +  1.0   Pyruvate_c0  => 1.0   ADP_c0  +  1.0   Phosphoenolpyruvate_c0  +  1.0   H+_c0  

R_rxn00459_c0 

## add exchange reactions from drift model

In [ ]:
import pandas as pd
medium = pd.read_excel('DSMZ_1409_media_KEGG.xls')


In [ ]:
exreaction = [rea for rea in drift_model.reactions if rea.id.startswith('EX')]
new_model.add_reactions(exreaction)

In [49]:
new_model

Name,Methylocystis_bryophila
Memory address,275d6ea92b0
Number of metabolites,1037
Number of reactions,921
Number of genes,0
Number of groups,0
Objective expression,1.0*rxnbiomass - 1.0*rxnbiomass_reverse_15500
Compartments,c0


## add reactions and metabolites note

https://narrative.kbase.us/#catalog/apps/kb_uploadmethods/import_file_as_fba_model_from_staging

In [12]:
import cobra
model = cobra.io.read_sbml_model(org_name+'_model\\'+org_name+'_add_from_modelseed.xml_model.xml')
model

'' is not a valid SBML 'SId'.
No objective coefficients in model. Unclear what should be optimized


Name,
Memory address,275c0232490
Number of metabolites,1140
Number of reactions,1129
Number of genes,0
Number of groups,0
Objective expression,0
Compartments,"c0, e0"


In [ ]:
# add gene
print('======================== \n genes of %d reactions begin to added \n'%i)
#mapping_reaction = pd.read_excel(org_name+'_model\\'+org_name+'_mapping_reaction.xlsx', index_col=0)  
i=1
for reaction in new_model.reactions:
    rxnid = [rxn for rxn in reaction.id.split('_') if rxn.startswith('rxn')]
    if rxnid:
        gene_list_str = '( '+mapping_reaction.query('ModelSeed_ID == "'+ rxnid[0] +'"')['entry_name'].values[0].replace(' ', ' or ').replace('mbry:', '')+' )'
        reaction.gene_reaction_rule = gene_list_str
    if i%100 == 0:
        print('%d / %d reactions genes added succesfully'%(i,len(new_model.reactions)))
    i+=1
print('%d / %d reactions genes added succesfully'%(len(new_model.reactions),len(new_model.reactions)))
